In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from sklearn.utils import class_weight
import numpy as np
from tensorflow.keras import layers, regularizers

### Loading the Data

In [2]:
cvd_data = pd.read_csv(r"Datasets\Analyzed\Analyzed_CVD_Prediction.csv")
cvd_data.head()

,heart_rate,oxygen_saturation,respiratory_rate,temperature,Label
0,93.0,98.0,39.0,36.0,1
1,97.0,98.0,16.0,36.5,0
2,82.0,97.0,25.0,36.3,1
3,98.0,96.0,16.0,36.5,1
4,98.0,99.0,28.0,37.2,1


# Training a DL Model using Keras with 2 Layers

In [3]:
# Split the data into training and validation sets
train_data = cvd_data.sample(frac=0.8, random_state=0)
val_data = cvd_data.drop(train_data.index)

# Preprocess the data
train_labels = train_data.pop('Label')
val_labels = val_data.pop('Label')

# Convert labels to numpy array if not already
train_labels_np = np.array(train_labels)

# Compute the class weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_labels_np),
    y=train_labels_np
)

# Convert to dictionary form
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
print("Class Weights:", class_weights_dict)

# Define the model
model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_data, train_labels, epochs=50,
                    validation_data=(val_data, val_labels),class_weight=class_weights_dict)

# Evaluate the model
test_loss, test_acc = model.evaluate(val_data, val_labels)

print('Test accuracy:', test_acc)


Class Weights: {0: 3.3002985074626867, 1: 0.5892761965675302}
Epoch 1/50
346/346 [==============================] - 5s 6ms/step - loss: 0.5141 - accuracy: 0.8262 - val_loss: 0.3859 - val_accuracy: 0.8647
Epoch 2/50
346/346 [==============================] - 2s 4ms/step - loss: 0.2440 - accuracy: 0.8755 - val_loss: 0.1860 - val_accuracy: 0.9215
Epoch 3/50
346/346 [==============================] - 1s 3ms/step - loss: 0.2079 - accuracy: 0.9009 - val_loss: 0.2442 - val_accuracy: 0.9128
Epoch 4/50
346/346 [==============================] - 1s 3ms/step - loss: 0.2156 - accuracy: 0.9027 - val_loss: 0.3054 - val_accuracy: 0.9128
Epoch 5/50
346/346 [==============================] - 1s 3ms/step - loss: 0.1933 - accuracy: 0.9105 - val_loss: 0.1674 - val_accuracy: 0.8940
Epoch 6/50
346/346 [==============================] - 1s 3ms/step - loss: 0.1978 - accuracy: 0.9103 - val_loss: 0.1923 - val_accuracy: 0.9034
Epoch 7/50
346/346 [==============================] - 1s 3ms/step - loss: 0.1924 - acc

In [ ]:
# Evaluate the model
test_loss, test_acc = model.evaluate(val_data, val_labels)
print('Test accuracy:', test_acc)

87/87 [==============================] - 0s 3ms/step - loss: 0.2262 - accuracy: 0.9197
Test accuracy: 0.9196816086769104


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

preds = (model.predict(val_data) > 0.5).astype(int)
print(confusion_matrix(val_labels, preds))
print(classification_report(val_labels, preds))

87/87 [==============================] - 1s 3ms/step
[[ 414    0]
 [ 222 2128]]
              precision    recall  f1-score   support

           0       0.65      1.00      0.79       414
           1       1.00      0.91      0.95      2350

    accuracy                           0.92      2764
   macro avg       0.83      0.95      0.87      2764
weighted avg       0.95      0.92      0.93      2764



#### These scores seems good to be deploy, so it will be converted into a TFLite model

### Converting this model to TFLite Model

In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]  # <-- ensure Pi compatibility
tflite_model = converter.convert()

with open('Models/CVD_Classification_Model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\SANDIN~1\AppData\Local\Temp\tmps48pi9a1\assets
